In [22]:
# 파일 업로드(드래그로 올린거랑 같음), API 파일 예제
# git에 api-key가 업로드되면 보안문제 발생하므로
#from google.colab import files
#uploaded = files.upload()
#filename = list(uploaded.keys())[0]
#filename

In [23]:
# API 키 가져오기
filename = '/content/카카오API.txt'
with open(filename) as f:
    api_key = f.read()
    
len(api_key)  # 32글자가 되야됨, 파일키 정상 문자열개수

32

In [24]:
# 온라인으로 보내거나 받아올 때 사용, 파이썬으로 웹사이트 연동할 때
import requests

# url에서 한글 코드를 변환
# %EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%EC%B2%AD = '서울특별시청'
from urllib.parse import quote

In [34]:
# 요청 정보 url 작성
# EX)
# curl -v -X GET "https://dapi.kakao.com/v2/local/search/address.json" \
#  -H "Authorization: KakaoAK ${REST_API_KEY}" \
#  --data-urlencode "query=전북 삼성동 100" 

addr = '서울시 종로구 종로1길 36(수송동)'
temp = quote(addr)
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
url = f'{search_url}?query={temp}'


In [38]:
# 정보 요청
# json() 가져오지 않는 경우는 txt
result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()

In [39]:
result

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12',
    'b_code': '1111012400',
    'h_code': '1111061500',
    'main_address_no': '146',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '종로1.2.3.4가동',
    'region_3depth_name': '수송동',
    'sub_address_no': '12',
    'x': '126.978988255925',
    'y': '37.5735051436739'},
   'address_name': '서울 종로구 종로1길 36',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 종로구 종로1길 36',
    'building_name': '종로구청',
    'main_building_no': '36',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_name': '수송동',
    'road_name': '종로1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.978988255925',
    'y': '37.5735051436739',
    'zone_no': '03152'},
   'x': '126.978988255925',
   'y': '37.5735051436739'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [44]:
result.keys()

dict_keys(['documents', 'meta'])

In [50]:
lon = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])

In [51]:
# .. 텍스트 위경도 삽입
import pandas as pd

df = pd.read_csv('/content/공공기관.csv')
df

,공공기관,도로명주소
0,종로구청,서울특별시 종로구 종로1길 36(수송동)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,양천구청,서울특별시 양천구 목동동로 105(신정동)


In [58]:
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list= [], []
for i in df.index:
    temp = quote(df.도로명주소[i])
    url = f'{search_url}?query={temp}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
    lon_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

In [59]:
lon_list, lat_list

([126.978988255925,
  127.056268317802,
  127.105859984389,
  126.903356973848,
  126.866542541936],
 [37.5735051436739,
  37.6545228397157,
  37.514477182474,
  37.5634263131969,
  37.5170753784215])

In [70]:
df['위도'] = lat_list
df['경도'] = lon_list
df.to_csv('공공기관_위경도추가.csv', index=False, encoding="cp949")  # 자동으로 인덱스가 계속생성
df

,공공기관,도로명주소,위도,경도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),37.573505,126.978988
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654523,127.056268
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.105860
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,양천구청,서울특별시 양천구 목동동로 105(신정동),37.517075,126.866543


# 지도에 정보 표시

In [64]:
# 지도에 정보 표시
import folium

In [71]:
map = folium.Map(location=[df.위도.mean(),df.경도.mean()], zoom_start=12)

for i in df.index:
    folium.Circle(radius=300,
                  location=[df.위도[i],df.경도[i]],
                  popup=folium.Popup(df.도로명주소[i], max_width=200),
                  tooltip=df.공공기관[i],
                  color='red',
                  fill=True,
                  fill_color='#3186cc').add_to(map)
title = '<h3 align="center" style="font-size:20"> 서울시 구청</h3>'
map.get_root().html.add_child(folium.Element(title))
map